In [3]:
import requests 
import json
import pprint as pp # pretty print

import aries_cloudagent.wallet.crypto as crypto
import base64
import base58
import uuid
import hashlib
import jupyter_util as util
from importlib import reload
reload(util)

<module 'jupyter_util' from '/home/sevni/Documents/thclab/aries/aries-acapy-plugin/controller_acapy/jupyter_util.py'>

In [4]:
def save_and_retrieve():
    add = requests.post('http://localhost:8150/pds/save', json={
      "payload": "testing"
    })
    print(add)
    add = json.loads(add.text)
    print(add)

    get = requests.get('http://localhost:8150/pds/' + str(add['payload_id']))
    print(get)
    get = json.loads(get.text)
    print(get)
        
        
def set_pds_config(pds_type, settings = {}):
    config = requests.post('http://localhost:8150/pds/settings', json={
      "settings": {
          pds_type: settings
      }
    })
    print(config)
    config = json.loads(config.text)
    print(config)
    
# set_pds_config("own_your_data", {
#     "client_id": "8SX1-RBazhic9ftG4HyG2XegrQ2kdLGu0hd-Ty3IZnE", 
#     "client_secret": "gEg9_tTh2V1ZK6piL9tZ38YO-1xqXLDXCccJS4uQUJg"
# })
# save_and_retrieve()
# set_pds_config("local")
# save_and_retrieve()
# set_pds_config("data_vault")
# save_and_retrieve()



## Enter Invite Url:

In [5]:
agent1 = {}
plugin = {}
endpoint = "http://localhost:8150/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent1['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent1['url'])

http://agent1.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiZjA2ZDFjZmQtNjM0MC00ZjMwLWJkZDEtM2Q2ZWQ0ZjY2OTZmIiwgInNlcnZpY2VFbmRwb2ludCI6ICJodHRwOi8vYWdlbnQxLmxvY2FsaG9zdCIsICJsYWJlbCI6ICJNYWluIChhZG1pbikiLCAicmVjaXBpZW50S2V5cyI6IFsiQmMxQU10QlhveXZlZXRIRmpmUkZBQ3drRnBLWENENFZXV0NCU0RTN21QMXoiXX0=


In [6]:
agent2 = {}
endpoint = "http://localhost:8151/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent2['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent2['url'])

http://agent2.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiZWIwNjYwNjQtNmFmYS00ODY3LTk2YjQtNDliZTMxYzE0YTMwIiwgInNlcnZpY2VFbmRwb2ludCI6ICJodHRwOi8vYWdlbnQyLmxvY2FsaG9zdCIsICJsYWJlbCI6ICJDbGllbnQgKGFkbWluKSIsICJyZWNpcGllbnRLZXlzIjogWyJESHVVeW04MnFDUUZpTTRIM1N3S0pyZkJoNkhhTHNlVWRzb1FFZ0RaVjRpUiJdfQ==


In [7]:
agent1 = util.processInviteUrl(agent1['url'])
agent2 = util.processInviteUrl(agent2['url'])
print("Decoded invitation:\n")
pp.pprint(agent2['invite'])

Decoded invitation:

{'@id': 'eb066064-6afa-4867-96b4-49be31c14a30',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
 'label': 'Client (admin)',
 'recipientKeys': ['DHuUym82qCQFiM4H3SwKJrfBh6HaLseUdsoQEgDZV4iR'],
 'serviceEndpoint': 'http://agent2.localhost'}


In [8]:
### Generating a did key using aries crypto module
# Create a key pair with random seed 
# Order: 0 - public key, 1 - secret / pritvate key
plugin['keypair'] = crypto.create_keypair()
# it seems to me that did is an encoded sub string of public key
# or maybe just some arbitrary random number
did = plugin['keypair'][0][:16]
# final version of our did
plugin['did'] = base58.b58encode(did).decode("ascii")
# encoding keys to base58
plugin['public_key_b58'] = \
    base58.b58encode(plugin['keypair'][0]).decode("ascii")
plugin['private_key_b58'] = \
    base58.b58encode(plugin['keypair'][1]).decode("ascii")

print("Private key: \n", plugin['private_key_b58'])
print("\nPublic key: \n", plugin['public_key_b58'])

Private key: 
 5yS1rGcWNRCTGprBHeGBfg1Jt2YU1r5s9RBFzozP29oxPUyyJ9d4T8CgP83pxLKJKD3yiWMGKu6PBb6asSNEq16X

Public key: 
 EWiSiezMNkrVV1Lhd5RHnWtg1SBxvACv9GYbS2ipujaX


**Sending a connection request to acapy**

In [9]:
agent1['connection'] = util.connectWithAcapy(agent1, plugin)
agent2['connection'] = util.connectWithAcapy(agent2, plugin)
# print("Decoded acapy response: \n")
# pp.pprint(agent1['conn'])

**Unpacking connection data embedded in the response**

In [10]:
agent1['connection_details'] = util.decodeConnectionDetails(agent1['connection'])
agent2['connection_details'] = util.decodeConnectionDetails(agent2['connection'])

**Adding the connection to connection list**

In [11]:
agents = [
    {
        "label": agent1['invite']['label'],
        "DIDDoc": agent1['connection_details']['DIDDoc'],
        "myKey": plugin['keypair'][1]
    },
    {
        "label": agent2['invite']['label'],
        "DIDDoc": agent2['connection_details']['DIDDoc'],
        "myKey": plugin['keypair'][1]
    },
]

# pp.pprint(connections)

In [12]:
connection = util.connectAgents(agents[0], agents[1])
connection[1] = connection[1]['raw_repr']['connection_id']
connection[0] = connection[0]['connection_id']
print(connection[0], "\n", connection[1])

0978411e-d461-4b45-ae38-cda5a554a9c3 
 206855a9-c266-41d1-95cf-1bbe356cdbb1


In [13]:
# add = requests.post('http://localhost:8150/test-vs', json={
#   "connection_id": connection[0]
# })
# print(add.text)
# add = json.loads(add.text)

### FEATURE DISCOVERY

In [14]:
# pp.pprint(util.getFeatureDiscovery(agents[0]))

In [15]:
# consentSchema: function() {
#       return {
#         namespace: "consent",
#         DRI: "fArVHJTQSKHu2CeXJocQmH3HHxzZXsuQD7kzyHJhQ49s"
#       }
#     },

# consentAnswers: function() {
#       return [
#         "zQmNeoZGfo1chKACu9RLrVoG4afjLrGnAz1rhsfvGZwg9Z3",
#         "zQmPsU57nqWY8jzndU9AE2RK4EXvjMLGmytVpUNxfRpm18G"
#       ]

In [16]:
add = requests.post('http://localhost:8151/verifiable-services/add', json={
  "label": "consent_test",
  "service_schema": {
    "oca_schema_namespace": "string",
    "oca_schema_dri": "string"
  },
  "consent_schema": {
    "oca_schema_namespace": "consent",
    "oca_schema_dri": "fArVHJTQSKHu2CeXJocQmH3HHxzZXsuQD7kzyHJhQ49s",
    "data_dri": "zQmPsU57nqWY8jzndU9AE2RK4EXvjMLGmytVpUNxfRpm18G"
  }
})
print(add.text)
add = json.loads(add.text)

{"service_schema": {"oca_schema_namespace": "string", "oca_schema_dri": "string"}, "consent_schema": {"data_dri": "zQmPsU57nqWY8jzndU9AE2RK4EXvjMLGmytVpUNxfRpm18G", "oca_schema_namespace": "consent", "oca_schema_dri": "fArVHJTQSKHu2CeXJocQmH3HHxzZXsuQD7kzyHJhQ49s"}, "updated_at": "2020-10-06 10:30:05.484373Z", "label": "consent_test", "created_at": "2020-10-06 10:30:05.484373Z"}


In [17]:


newDid1 = util.createDid(agents[1])
newDid0 = util.createDid(agents[0])

try:
    util.registerDidOnLedger(agents[1], newDid1['result']['did'], newDid1['result']['verkey'])
except:
    pass
try:
    util.registerDidOnLedger(agents[0], newDid0['result']['did'], newDid0['result']['verkey'])
except:
    pass

util.setPublicDid(agents[1], newDid1['result']['did'])
util.setPublicDid(agents[0], newDid0['result']['did'])

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/did',
 '@id': '97897954-f07d-4193-a1c0-31cd0af9f3e1',
 '~thread': {'thid': 'ca99369e-9a3e-46f1-9e89-75a9a40f50ad'},
 'result': {'did': 'Xi4nWQLWcexqrrMxsoQT6Y',
  'verkey': 'Hjjz4Rb3Rdr7BkEAiCAPH45ZY9vwQJZPL4CfNpyCnBjt',
  'metadata': {'public': True, 'posted': True}}}

In [18]:
service_list = requests.get('http://localhost:8150/verifiable-services/DEBUGrequest/' + connection[0])
print(service_list)
service_list = json.loads(service_list.text)
pp.pprint(service_list)

<Response [200]>
{'connection_id': '0978411e-d461-4b45-ae38-cda5a554a9c3',
 'created_at': '2020-10-06 10:30:10.809041Z',
 'services': [{'consent_schema': {'data_dri': 'zQmPsU57nqWY8jzndU9AE2RK4EXvjMLGmytVpUNxfRpm18G',
                                  'oca_schema_dri': 'fArVHJTQSKHu2CeXJocQmH3HHxzZXsuQD7kzyHJhQ49s',
                                  'oca_schema_namespace': 'consent'},
               'label': 'consent_test',
               'service_id': 'f10889c2-c8b1-4608-bff8-15b0d50f19e6',
               'service_schema': {'oca_schema_dri': 'string',
                                  'oca_schema_namespace': 'string'}}],
 'updated_at': '2020-10-06 10:30:10.809041Z'}


In [19]:
apply = requests.post('http://localhost:8150/verifiable-services/apply', json={
  "connection_id": connection[0],
  "service_id": service_list['services'][0]['service_id'],
  "payload": "abc",
  "service": service_list['services'][0],
})
apply = json.loads(apply.text)
print(apply)

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/verifiable-services/1.0/application', '@id': '11b3757f-007c-471f-bfbd-87473184ad27', 'data_dri': 'b1fb1f3a6634d2271a21e2dcc012153fba6a6e4de7772c2fe86748ac36cab770', 'exchange_id': '0c01cb87-e576-4a89-833d-08e342c8385d', 'credential_definition_id': 'Xi4nWQLWcexqrrMxsoQT6Y:3:CL:1299:consent_schema', 'service_consent_match_id': 'd73fe490-6117-4008-a0d5-fecf018a058a', 'service_id': 'f10889c2-c8b1-4608-bff8-15b0d50f19e6'}


In [22]:
issue_self = requests.post('http://localhost:8151/verifiable-services/get-issue-self', json={
    "exchange_id": apply['exchange_id'],
})
print(issue_self)
issue_self = json.loads(issue_self.text)
pp.pprint(issue_self)

issue_self = requests.post('http://localhost:8151/verifiable-services/process-application', json={
    "issue_id": issue_self[0]['issue_id'],
    "decision": "accept",
})
print(issue_self)
issue_self = json.loads(issue_self.text)
pp.pprint(issue_self)

<Response [200]>
[{'author': 'other',
  'connection_id': '206855a9-c266-41d1-95cf-1bbe356cdbb1',
  'consent_schema': '{"oca_schema_namespace": "consent", "oca_schema_dri": '
                    '"fArVHJTQSKHu2CeXJocQmH3HHxzZXsuQD7kzyHJhQ49s", '
                    '"data_dri": '
                    '"zQmPsU57nqWY8jzndU9AE2RK4EXvjMLGmytVpUNxfRpm18G"}',
  'created_at': '2020-10-06 10:30:19.937292Z',
  'exchange_id': '0c01cb87-e576-4a89-833d-08e342c8385d',
  'issue_id': '51413a470912cf95cf66c1d58c71c73e7bc15cc0c30ff894e707c24d00489942',
  'label': 'consent_test',
  'payload_dri': 'aa8bf3ec-308c-4db5-8b91-9564d62069d0',
  'service_consent_match_id': 'd73fe490-6117-4008-a0d5-fecf018a058a',
  'service_id': 'f10889c2-c8b1-4608-bff8-15b0d50f19e6',
  'service_schema': '{"oca_schema_namespace": "string", "oca_schema_dri": '
                    '"string"}',
  'state': 'accepted',
  'updated_at': '2020-10-06 10:30:50.874458Z'}]


In [23]:
credentials = requests.get('http://localhost:8151/credentials')
credentials = json.loads(credentials.text) 
pp.pprint(credentials)

{'results': [{'context': ['https://www.w3.org/2018/credentials/v1',
                          'https://www.w3.org/2018/credentials/examples/v1'],
              'created_at': '2020-10-06 10:30:20.207798Z',
              'credentialSubject': {'data_dri': 'zQmPsU57nqWY8jzndU9AE2RK4EXvjMLGmytVpUNxfRpm18G',
                                    'oca_schema_dri': 'fArVHJTQSKHu2CeXJocQmH3HHxzZXsuQD7kzyHJhQ49s',
                                    'oca_schema_namespace': 'consent',
                                    'service_consent_match_id': 'd73fe490-6117-4008-a0d5-fecf018a058a'},
              'id': 'Xi4nWQLWcexqrrMxsoQT6Y',
              'issuer': 'https://example.edu/issuers/565049',
              'proof': {'jws': 'FEDOXLIh2lao+qqFJi91JfPC0F+Yz7xZmb4enBORqY1b0jYCEFX1CYLQLtq6Bl8odZGREtL0tHiKEKqyJcY6BA'},
              'type': ['VerifiableCredential', 'ConsentCredential'],
              'updated_at': '2020-10-06 10:30:20.207798Z'}]}


In [24]:
proof_schema = {
  "proof_request": {
    "version": "1.0",
    "name": "Proof request",
    "requested_attributes": {
      "data_dri": {
        "non_revoked": {
          "from_epoch": 1500000000,
          "to_epoch": 1700758817
        },
        "restrictions": [
          {
            "cred_def_id": apply['credential_definition_id'],
          }
        ],
        "name": "data_dri"
      },
    },
    "requested_predicates": {
    }
  },
  "connection_id": connection[1],
#   "trace": True,
  "comment": "string",
  "nonce": "1234567890"
}
proof_json = json.dumps(proof_schema)

proof = requests.post('http://localhost:8151/present-proof/send-request', json=proof_schema)
print(proof.text)

500 Internal Server Error

Server got itself in trouble


In [359]:
proof = requests.get('http://localhost:8150/present-proof/records')
proof_requests = json.loads(proof.text)['results']
proof_request = proof_requests[-4]
pp.pprint(proof_request)

{'connection_id': '994ebd0f-14c2-428c-be53-e730bc3ca7a7',
 'created_at': '2020-09-22 11:58:22.954779Z',
 'initiator': 'external',
 'presentation_exchange_id': '964c067c-46fb-4a4f-b30d-253029bf6d53',
 'presentation_request': {'name': 'Proof request',
                          'nonce': '236523038637111089936187',
                          'requested_attributes': {'data_dri': {'name': 'data_dri',
                                                                'non_revoked': {'from_epoch': 1500000000,
                                                                                'to_epoch': 1700758817},
                                                                'restrictions': [{'cred_def_id': 'WWZtMuY1V9hvFF1ezey2EM:3:CL:1175:Services'}]}},
                          'requested_predicates': {},
                          'version': '1.0'},
 'role': 'prover',
 'state': 'request_received',
 'thread_id': 'aab467d6-30db-442a-a23e-354f77100467',
 'trace': False,
 'updated_at': '2020-09-22 

In [360]:
url = 'http://localhost:8150/present-proof/records/' + \
       proof_request['presentation_exchange_id'] + \
       '/send-presentation'

presentation_schema = {
  "requested_attributes": {
    "data_dri": {
      "cred_id": credentials['results'][0]['referent'],
        "revealed": True
    },
  },
   "requested_predicates": {
  },
  "self_attested_attributes": {
  }
}
print(presentation_schema)
present = requests.post(url, json=presentation_schema)

print(present.text)

{'requested_attributes': {'data_dri': {'cred_id': '657ec1be-2d72-41ba-a1c2-725082d1c759', 'revealed': True}}, 'requested_predicates': {}, 'self_attested_attributes': {}}
{"initiator": "external", "presentation_exchange_id": "964c067c-46fb-4a4f-b30d-253029bf6d53", "created_at": "2020-09-22 11:58:22.954779Z", "trace": false, "state": "presentation_sent", "connection_id": "994ebd0f-14c2-428c-be53-e730bc3ca7a7", "presentation_request": {"version": "1.0", "name": "Proof request", "requested_attributes": {"data_dri": {"non_revoked": {"from_epoch": 1500000000, "to_epoch": 1700758817}, "restrictions": [{"cred_def_id": "WWZtMuY1V9hvFF1ezey2EM:3:CL:1175:Services"}], "name": "data_dri"}}, "requested_predicates": {}, "nonce": "236523038637111089936187"}, "thread_id": "aab467d6-30db-442a-a23e-354f77100467", "updated_at": "2020-09-22 11:58:42.143421Z", "presentation": {"proof": {"proofs": [{"primary_proof": {"eq_proof": {"revealed_attrs": {"data_dri": "46796680751272472783988362602137825638383598079

In [53]:
send = requests.post('http://localhost:8151/schema-exchange/request-schema', json={
  "hash_id": add['hash_id'],
  "connection_id": connection[1]
})

In [54]:
send = requests.post('http://localhost:8151/schema-exchange/request-schema', json={
  "hash_id": "ddd",
  "connection_id": connection[1]
})

In [55]:
print(send.text)

{"hash_id": "ddd", "connection_id": "a361d994-aaea-4f5c-9bf4-9644a774de2b", "record_id": "1e61b917-946b-4a56-8810-5b48081f79e6", "exchange_id": "c5f71769-b806-49fe-8861-15f49ffad9e3", "state": "pending"}


### GET Request By RECORD_ID

### GET Request List and filter by state and author

In [56]:
request_list = requests.post('http://localhost:8150/schema-storage/debug/request/list', json={})
request_list = json.loads(request_list.text)
print("Agent1:", )
pp.pprint(request_list)
request_list = requests.post('http://localhost:8151/schema-storage/debug/request/list', json={})
request_list = json.loads(request_list.text)
print("\nAgent2:")
pp.pprint(request_list)

Agent1:
[{'author': 'other',
  'connection_id': '844f6e7f-c13e-4ef8-b018-754fc5205445',
  'created_at': '2020-07-16 09:09:17.991960Z',
  'exchange_id': 'c5f71769-b806-49fe-8861-15f49ffad9e3',
  'payload': 'ddd',
  'state': 'failed to find the record',
  'updated_at': '2020-07-16 09:09:17.991960Z'},
 {'author': 'other',
  'connection_id': '844f6e7f-c13e-4ef8-b018-754fc5205445',
  'created_at': '2020-07-16 09:09:16.748526Z',
  'exchange_id': 'f51e4a06-2214-4b9e-a28a-d4c336258f66',
  'payload': '9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08',
  'state': 'approved',
  'updated_at': '2020-07-16 09:09:16.748526Z'}]

Agent2:
[{'author': 'self',
  'connection_id': 'a361d994-aaea-4f5c-9bf4-9644a774de2b',
  'created_at': '2020-07-16 09:09:17.950785Z',
  'exchange_id': 'c5f71769-b806-49fe-8861-15f49ffad9e3',
  'payload': 'ddd',
  'state': 'failed to find the record',
  'updated_at': '2020-07-16 09:09:18.029400Z'},
 {'author': 'self',
  'connection_id': 'a361d994-aaea-4f5c-9bf4-

### GET RECORD LIST

In [57]:
getRequests = requests.post('http://localhost:8150/schema-storage/list')
record_list = json.loads(getRequests.text)
print("Agent1: ")
pp.pprint(record_list)

getRequests = requests.post('http://localhost:8151/schema-storage/list')
record_list = json.loads(getRequests.text)
print("Agent2: ")
pp.pprint(record_list)

Agent1: 
[{'author': 'self',
  'created_at': '2020-07-16 09:09:15.407083Z',
  'hash_id': '9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08',
  'payload': 'test',
  'updated_at': '2020-07-16 09:09:15.407083Z'}]
Agent2: 
[{'author': 'a361d994-aaea-4f5c-9bf4-9644a774de2b',
  'created_at': '2020-07-16 09:09:16.808608Z',
  'hash_id': '9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08',
  'payload': 'test',
  'updated_at': '2020-07-16 09:09:16.808608Z'}]


In [ ]:

# message = util.buildMessage(
#         'https://didcomm.org/issue-credential/1.0/propose-credential'
#     )
# testing = util.sendMessage(message, agents[0])
# pp.pprint(testing)

In [ ]:
# pp.pprint(util.sendSchema(agents[0], "schesssdddvd", "1.0", ["memesssaaaaaaa"]))

In [ ]:
# result = util.schemaGetList(agents[0])
# last_schema_id = result['results'][-1]['schema_id']
# print(result)

In [ ]:
# util.schemaGet(agents[0], last_schema_id)